# Influenza A - all subtypes


### NCBI entrez

I initially attempted to label all segments using the annotations in the genbank file that I could obtain using Biopython's entrez tool - but this is very slow. It takes over 2min to download 100k records, and Influenza A has over 1M records. Additionally, the annotations are often misspelled and are not standardized - so I ended up with long lists of protein synonyms that were still not complete (see the attached config). Ultimately this lead to only ~110k sequences being annotated with a segment. 

### Blast

Attempt to use Blast to label sequences segments and subtypes using local alignment: 

```
makeblastdb -in influenza_segments.fasta -dbtype nucl -out influenza_db
blastn -query ../results/ncbi_dataset/data/genomic.fna  -db influenza_db -out results.txt -outfmt 6 #only results
blastn -query ../results/ncbi_dataset/data/genomic.fna  -db influenza_db -out results.txt -outfmt 7 #results and sequences with no hits
```

This leads to over 80k sequences that have no hits (see field at the bottom). I tried to tune the query to increase the number of hits, but this was not successful: 

```
blastn -query ../results/ncbi_dataset/data/genomic.fna  -db influenza_db -out results.txt -outfmt 6 -evalue 1e5 -max_target_seqs 1 -dust no -word_size 7 -max_hsps 1
```

In [14]:
from Bio import Entrez, SeqIO

# Set your email address (required by NCBI)
Entrez.email = "your_email@example.com"

for seg, accession_number in enumerate(["NC_026438.1", "NC_026435.1", "NC_026437.1", "NC_026433.1", "NC_026436.1", "NC_026434.1", "NC_026431.1", "NC_026432.1"]):

    # Fetch the sequence in FASTA format
    with Entrez.efetch(db="nucleotide", id=accession_number, rettype="fasta", retmode="text") as handle:
        record = SeqIO.read(handle, "fasta")
        print(f">H1N1_segment_{seg+1}")
        print(record.seq)

for seg, accession_number in enumerate(["NC_026422.1", "NC_026423.1", "NC_026424.1", "NC_026425.1", "NC_026426.1", "NC_026429.1", "NC_026427.1", "NC_026428.1"]):

    # Fetch the sequence in FASTA format
    with Entrez.efetch(db="nucleotide", id=accession_number, rettype="fasta", retmode="text") as handle:
        record = SeqIO.read(handle, "fasta")
        print(f">H7N9_segment_{seg+1}")
        print(record.seq)

for seg, accession_number in enumerate(["NC_007378.1", "NC_007375.1", "NC_007376.1", "NC_007374.1", "NC_007381.1", "NC_007382.1", "NC_007377.1", "NC_007380.1"]):

    # Fetch the sequence in FASTA format
    with Entrez.efetch(db="nucleotide", id=accession_number, rettype="fasta", retmode="text") as handle:
        record = SeqIO.read(handle, "fasta")
        print(f">H2N2_segment_{seg+1}")
        print(record.seq)

for seg, accession_number in enumerate(["NC_007373.1", "NC_007372.1", "NC_007371.1", "NC_007366.1", "NC_007369.1", "NC_007368.1", "NC_007367.1", "NC_007370.1"]):

    # Fetch the sequence in FASTA format
    with Entrez.efetch(db="nucleotide", id=accession_number, rettype="fasta", retmode="text") as handle:
        record = SeqIO.read(handle, "fasta")
        print(f">H3N2_segment_{seg+1}")
        print(record.seq)

for seg, accession_number in enumerate(["NC_004910.1", "NC_004911.1", "NC_004912.1", "NC_004908.1", "NC_004905.2", "NC_004909.1", "NC_004907.1", "NC_004906.1"]):

    # Fetch the sequence in FASTA format
    with Entrez.efetch(db="nucleotide", id=accession_number, rettype="fasta", retmode="text") as handle:
        record = SeqIO.read(handle, "fasta")
        print(f">H9N2_segment_{seg+1}")
        print(record.seq)


>H1N1_segment1
ATGGAGAGAATAAAAGAACTGAGAGATCTAATGTCGCAGTCCCGCACTCGCGAGATACTCACTAAGACCACTGTGGACCATATGGCCATAATCAAAAAGTACACATCAGGAAGGCAAGAGAAGAACCCCGCACTCAGAATGAAGTGGATGATGGCAATGAGATACCCAATTACAGCAGACAAGAGAATAATGGACATGATTCCAGAGAGGAATGAACAAGGACAAACCCTCTGGAGCAAAACAAACGATGCTGGATCAGACCGAGTGATGGTATCACCTCTGGCCGTAACATGGTGGAATAGGAATGGCCCAACAACAAGTACAGTTCATTACCCTAAGGTATATAAAACTTATTTCGAAAAGGTCGAAAGGTTGAAACATGGTACCTTCGGCCCTGTCCACTTCAGAAATCAAGTTAAAATAAGGAGGAGAGTTGATACAAACCCTGGCCATGCAGATCTCAGTGCCAAGGAGGCACAGGATGTGATTATGGAAGTTGTTTTCCCAAATGAAGTGGGGGCAAGAATACTGACATCAGAGTCACAGCTGGCAATAACAAAAGAGAAGAAAGAAGAGCTCCAGGATTGTAAAATTGCTCCCTTGATGGTGGCGTACATGCTAGAAAGAGAATTGGTCCGTAAAACAAGGTTTCTCCCAGTAGCCGGCGGAACAGGCAGTGTTTATATTGAAGTGTTGCACTTAACCCAAGGGACGTGCTGGGAGCAGATGTACACTCCAGGAGGAGAAGTGAGAAATGATGATGTTGACCAAAGTTTGATTATCGCTGCTAGAAACATAGTAAGAAGAGCAGCAGTGTCAGCAGACCCATTAGCATCTCTCTTGGAAATGTGCCACAGCACACAGATTGGAGGAGTAAGGATGGTGGACATCCTTAGACAGAATCCAACTGAGGAACAAGCCGTAGACATATGCAAGGCAGCAATAGGGTTGAGGATTAGCTCATCTTTCAGTTTTGGTGGGTTCA

In [27]:
from Bio import SeqIO

# Extract IDs from input FASTA
fasta_ids = {}

for record in SeqIO.parse("results/ncbi_dataset/data/genomic.fna", "fasta"):
    fasta_ids[record.id] = record.description

# Extract IDs from BLAST output
blast_ids = []
with open("segments/results.txt", "r") as blast_file:
    for line in blast_file:
        blast_ids.append(line.split("\t")[0])

print(blast_ids[0])
print(fasta_ids[blast_ids[0]])

print(len(fasta_ids.keys()))

# Find IDs with no hits
no_hits_ids = set(fasta_ids.keys()) - set(blast_ids)
print(len(no_hits_ids))

# Save IDs without hits
with open("no_hits_ids.txt", "w") as output_file:
    for no_hit_id in no_hits_ids:
        output_file.write(fasta_ids[no_hit_id] + "\n")

MH828535.1
MH828535.1 Influenza A virus (A/Viet nam/2548/2014(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
1136048
81284


## Nextclade sort

Cornelius then showed me nextclade sort - which is similar to Blast but faster and more customizable. It requires a minimizer_index file - which can be generated from the influenza reference sequences file I created above using the same code used by nextclade datasets (https://github.com/nextstrain/nextclade_data/blob/master/scripts/lib/minimizer.py).

Then the parameters can be tuned: 

```
nextclade sort -m influenza_segments.minimizer.json -r out.tsv  sample_influenza.fasta --max-score-gap 0.5 --min-score 0.05 --min-hits 1  --all-matches
```

Decreasing the min-score to 0.01 leads to each sequence matching with a segment but the quality of the matches is often poor and in the few cases where I have ncbi annotations there is an indication that the alignment is not accurate.

In [2]:
import copy

import numpy as np
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
import json

# Increment this version, if there are changes in the algorithm.
# Backwards compatibility must be ensured to not break client code: all versions must be computed and reffered to
# from the dataset's server 'index.json' file.
MINIMIZER_ALGO_VERSION = "1"

# Increment this version, if there are changes in the layout of the output file.
MINIMIZER_JSON_SCHEMA_VERSION = "3.0.0"

# What is this?
MAGIC_NUMBER_K = 17

# minimizer cutoff. The max is 1<<32 - 1, so with 28 uses roughly 1/16 of all kmers
CUTOFF = 1 << 32 - 1


# from lh3
def invertible_hash(x):
    m = (1 << 32) - 1
    x = (~x + (x << 21)) & m
    x = x ^ (x >> 24)
    x = (x + (x << 3) + (x << 8)) & m
    x = x ^ (x >> 14)
    x = (x + (x << 2) + (x << 4)) & m
    x = x ^ (x >> 28)
    x = (x + (x << 31)) & m
    return x


# turn a kmer into an integer
def get_hash(kmer):
    x = 0
    j = 0
    for i, nuc in enumerate(kmer):
        if i % 3 == 2:
            continue  # skip every third nucleotide to pick up conserved patterns
        if nuc not in "ACGT":
            return CUTOFF + 1  # break out of loop, return hash above cutoff
        else:  # A=11=3, C=10=2, G=00=0, T=01=1
            if nuc in "AC":
                x += 1 << j
            if nuc in "AT":
                x += 1 << (j + 1)
        j += 2

    return invertible_hash(x)


def get_ref_search_minimizers(seq: SeqRecord, k=MAGIC_NUMBER_K):
    seq_str = preprocess_seq(seq)
    minimizers = []
    # we know the rough number of minimizers, so we can pre-allocate the array if needed
    for i in range(len(seq_str) - k):
        kmer = seq_str[i : i + k]
        mhash = get_hash(kmer)
        if (
            mhash < CUTOFF
        ):  # accept only hashes below cutoff --> reduces the size of the index and the number of look-ups
            minimizers.append(mhash)
    return np.unique(minimizers)


def make_ref_search_index(refs):
    # collect minimizers for each reference sequence first
    minimizers_by_reference = list()
    for name, ref in sorted(refs.items()):
        minimizers = get_ref_search_minimizers(ref)
        minimizers_by_reference.append(
            {
                "minimizers": minimizers,
                "meta": {"name": name, "length": len(ref.seq), "nMinimizers": len(minimizers)},
            }
        )

    # construct an index where each minimizer maps to the references it contains via a bit set (here boolean np array)
    index = {"minimizers": {}, "references": []}
    for ri, minimizer_set in enumerate(minimizers_by_reference):
        for m in minimizer_set["minimizers"]:
            if m not in index["minimizers"]:
                index["minimizers"][m] = []
            index["minimizers"][m].append(ri)

        # reference will be a list in same order as the bit set
        index["references"].append(minimizer_set["meta"])

    normalization = np.array([x["length"] / x["nMinimizers"] for x in index["references"]])

    return {
        "schemaVersion": MINIMIZER_JSON_SCHEMA_VERSION,
        "version": MINIMIZER_ALGO_VERSION,
        "params": {
            "k": MAGIC_NUMBER_K,
            "cutoff": CUTOFF,
        },
        **index,
        "normalization": normalization,
    }


def preprocess_seq(seq: SeqRecord) -> str:
    return str(seq.seq).upper().replace("-", "")


def serialize_ref_search_index(index):
    index = copy.deepcopy(index)
    index["minimizers"] = {str(k): v for k, v in index["minimizers"].items()}
    index["normalization"] = index["normalization"].tolist()
    return index


def to_bitstring(arr) -> str:
    return "".join([str(x) for x in arr])


# Load the reference sequences
refs = {rec.id: rec for rec in SeqIO.parse("segments/influenza_segments.fasta", "fasta")}

index = make_ref_search_index(refs)

serialized = serialize_ref_search_index(index)

with open("influenza_segments.minimizer.json", "w") as f:
    json.dump(serialized, f)

In [40]:
import pandas as pd

# Load the TSV file
file_path = 'segments/out.tsv'
df = pd.read_csv(file_path, sep='\t')
df = df.astype({'index':'int'})

df = df.dropna(subset=['score'])

print(df['index'].sort_values().head())

print(df['index'].astype(int).max())

# Find the expected range of indexes
expected_indexes = set(range(df['index'].min(), df['index'].max() + 1))

# Find the actual indexes in the file
actual_indexes = set(df['index'].dropna().unique())

# Find missing indexes
missing_indexes = sorted(expected_indexes - actual_indexes)

# Output missing indexes
print(f"Number of missing indexes: {len(missing_indexes)}")
print("Missing indexes:", missing_indexes[:10], "...")

1      0
0      1
2      2
115    3
3      4
Name: index, dtype: int64
1136047
Number of missing indexes: 43276
Missing indexes: [8, 97, 102, 117, 118, 119, 124, 128, 151, 152] ...


In [43]:
import re


def extract_subtype(seq_name):
    pattern = r'\bH([1-9]|1[0-8])N([1-9]|1[0-2])\b'  # Match H1-18 and N1-12
    match = re.search(pattern, seq_name)
    return match.group(0) if match else np.nan

def extract_segment(seq_name):
    pattern = r'segment ([1-8])'
    match = re.search(pattern, seq_name)
    return match.group(0) if match else np.nan

def extract_accession(seq_name):
    return seq_name.split(' ')[0]

def extract_computed_subtype(dataset):
    return dataset.split('_')[0]

def extract_computed_segment(dataset):
    return ' '.join(dataset.split('_')[-2:])


file_path = 'segments/out.tsv'
df = pd.read_csv(file_path, sep='\t', dtype={'index': 'Int64'})

# Ensure 'score' is treated as a numeric column (replace 'score' with the actual column name if different)
df['score'] = pd.to_numeric(df['score'], errors='coerce')

# Drop rows where 'score' is NaN (optional, if there are invalid scores)
df = df.dropna(subset=['score'])

# Group by 'index', then sort within each group by 'score' and keep the highest score
df_sorted = df.sort_values(['index', 'score'], ascending=[True, False])
df_highest_per_group = df_sorted.drop_duplicates(subset='index', keep='first')

df_highest_per_group['ncbiSubType'] = df_highest_per_group['seqName'].apply(extract_subtype)
df_highest_per_group['ncbiSegment'] = df_highest_per_group['seqName'].apply(extract_segment)
df_highest_per_group['seqName'] = df_highest_per_group['seqName'].apply(extract_accession)
df_highest_per_group['inferredSubType'] = df_highest_per_group['dataset'].apply(extract_computed_subtype)
df_highest_per_group['inferredSegment'] = df_highest_per_group['dataset'].apply(extract_computed_segment)
df_highest_per_group.drop(columns='dataset', inplace=True)

# Reset index and save the result (optional)
df_highest_per_group.reset_index(drop=True, inplace=True)

# Optionally save to a new file
output_path = 'results/nextclade_sort.tsv'
df_highest_per_group.to_csv(output_path, sep='\t', index=False)

/var/folders/q5/yqfb2rjs2m50f2c00h3lxghh0000gr/T/ipykernel_84904/1895326991.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_highest_per_group['ncbiSubType'] = df_highest_per_group['seqName'].apply(extract_subtype)
/var/folders/q5/yqfb2rjs2m50f2c00h3lxghh0000gr/T/ipykernel_84904/1895326991.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_highest_per_group['ncbiSegment'] = df_highest_per_group['seqName'].apply(extract_segment)
/var/folders/q5/yqfb2rjs2m50f2c00h3lxghh0000gr/T/ipykernel_84904/189

In [44]:
non_null_count = df_highest_per_group['ncbiSegment'].notna().sum()
print(f"Number of non-null rows in column 'ncbiSegment': {non_null_count}")

filtered_df = df_highest_per_group[(df_highest_per_group['inferredSegment'] != df_highest_per_group['ncbiSegment']) & (df_highest_per_group['ncbiSegment'].notna())]

# Count the number of such rows
count_mismatch = len(filtered_df)

# Display the count
print(f"Number of rows where 'inferredSegment' is not equal to 'ncbiSegment' and 'ncbiSegment' is not null: {count_mismatch}")
print(filtered_df)

Number of non-null rows in column 'ncbiSegment': 882784
Number of rows where 'inferredSegment' is not equal to 'ncbiSegment' and 'ncbiSegment' is not null: 16
           index     seqName     score  numHits ncbiSubType ncbiSegment  \
164828    171359  MH718294.2  0.541375    264.0        H9N2   segment 6   
194911    202227  EF178516.1  0.510702    571.0        H5N1   segment 3   
295855    307209  KR824578.1  0.357671    284.0         NaN   segment 4   
351386    364722  MH718296.2  0.621351    303.0        H9N2   segment 6   
437300    454386  MH718298.2  0.602895    294.0        H9N2   segment 6   
507952    527598  MH718293.2  0.555730    271.0        H9N2   segment 6   
507953    527599  MH718295.2  0.572135    279.0        H9N2   segment 6   
685857    713038  KY241427.1  0.053453     46.0        H6N8   segment 6   
752848    782405  AF213911.1  0.356661     35.0        H3N2   segment 8   
753053    782624  MH718300.2  0.613149    299.0        H9N2   segment 6   
779482    810317

Now create auspice trees for each segment, with labeled subtypes (annotated and ncbi annotated), then use:
```
for i in {1..8}; do   
seqkit sample -n 1000 --out-file ../results/sample_segment_${i}.fasta ../results/segment_${i}.fasta
augur align --sequences ../results/sample_segment_${i}.fasta --output ../results/segment_${i}_aligned.fasta 
augur tree --alignment ../results/segment_${i}_aligned.fasta --output ../results/segment_${i}.nwk 
augur refine --tree ../results/segment_${i}.nwk --output-tree ../results/segment_${i}_refined.nwk  
augur traits --tree ../results/segment_${i}_refined.nwk \
            --metadata ../results/nextclade_sort.tsv \
            --output-node-data ../results/segment${i}_traits.json \
            --columns "inferredSegment inferredSubType ncbiSegment ncbiSubtype" \
            --metadata-id-columns seqName 
augur export v2 \
            --tree ../results/segment_${i}_refined.nwk \
            --metadata ../results/nextclade_sort.tsv \
            --node-data ../results/segment${i}_traits.json \
            --output ../results/segment${i}_auspice.json \
            --metadata-id-columns "seqName"  --auspice-config auspice_config.json
done 
```
to plot as an auspice tree

In [59]:
segment_lookup = {}
for _, row in df_highest_per_group.iterrows():
    segment_lookup[row['seqName']] = row['inferredSegment']

# Create file handles for each segment
output_files = {
    segment: open(f"results/segment_{segment}.fasta", "w", encoding="utf-8")
    for segment in range(1, 9)
}

try:
    # Parse the genomic file once
    with open("results/ncbi_dataset/data/genomic.fna", encoding="utf-8") as f_in:
        for record in SeqIO.parse(f_in, "fasta"):
            inferred_segment = segment_lookup.get(record.id)
            if inferred_segment:
                segment_number = int(inferred_segment.split()[1])
                if 1 <= segment_number <= 8:
                    output_files[segment_number].write(f">{record.id}\n{record.seq}\n")
finally:
    # Ensure all files are closed properly
    for file in output_files.values():
        file.close()